In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
from ipywidgets import widgets
import plotly
import pandas as pd
import os
import plotly.graph_objects as go
from IPython.display import display
import plotly.express as px
from IPython.display import HTML

In [5]:
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

In [6]:
class factionLocations:
    def __init__(self,referenceFaction):
        self.faction=referenceFaction  
        self.presentSystems=[]
        #self.listFactionLocations()
    def listFactionLocations(self):
        self.factionLocations=[]
        EDDBfactions="https://elitebgs.app/api/ebgs/v4/factions"
        URL =os.path.join(EDDBfactions)
        PARAMS = {"name":self.faction} 
        r = requests.get(url = URL, params = PARAMS) 
        data = r.json()
        for item in data["docs"]:
            for system in item["faction_presence"]:
                self.factionLocations.append( system["system_name"])
        #print("The faction %s is located in the following systems:"%self.faction)
        #for item in self.factionLocations:
        #    print(item)

        return self.factionLocations#, data["docs"]

In [7]:
class factionInfluence:
    def __init__(self,system,graph=0):
        #self.faction=referenceFaction
        #self.presentSystems=factionLocations(referenceFaction)
        self.influences={}
        self.influences["factions"]=[]
        self.influences["inf"]=[]
        self.influences["state"]=[]
        self.influences["factionState"]=[]
        self.influences["controllingState"]=[]
        self.system=system
        self.graph=graph
        #self.getInf()

    def getInf(self):
        endpoint="https://www.edsm.net/api-system-v1/factions"
        PARAMS = {"systemName":self.system} 
        r = requests.get(url=endpoint, params = PARAMS) 
        data = r.json()
        self.influences["controlling"]=data["controllingFaction"]["name"]
        
        #print(data["controllingFaction"])
        for item in data["factions"]:
            self.influences["factions"].append(item["name"])
            self.influences["inf"].append(item["influence"]*100)
            self.influences["state"].append("State : "+item["state"])
            self.influences["factionState"].append(item["name"]+" :: State: "+item["state"])
            if item["name"]==self.influences["controlling"]:
                self.influences["controllingInfo"]=data["controllingFaction"]["name"] + " :: "+ item["state"] + " :: "+ data["controllingFaction"]["allegiance"] + " :: "+ data["controllingFaction"]["government"]
        
        if self.graph:
            fig = go.Figure(
                data=[go.Bar(x=self.influences["factions"],y=self.influences["inf"],text=self.influences["state"],textposition='auto')],
                layout_title_text="%s :: Controlled by %s"%(self.system,self.influences["controllingInfo"])
            )
            fig.show()
        else:
            for key,value in self.influences.items():
                print(key,value)


In [8]:
class factionCoordinates:
    def __init__(self,faction,systems):
        self.faction=faction
        locations=factionLocations(referenceFaction=self.faction)
        self.systems=systems
        self.coordinates=[]
    def gatherCoordinates(self):
        for system in self.systems:
            endpoint="https://www.edsm.net/api-v1/system"
            PARAMS = {"systemName":system,"showCoordinates":1} 
            r = requests.get(url=endpoint, params = PARAMS) 
            data = r.json()
            #print(data)
            thisSystem=[]
            thisSystem.append(data["name"])
            thisSystem.append(data["coords"]["x"])
            thisSystem.append(data["coords"]["y"])
            thisSystem.append(data["coords"]["z"])
            thisSystem.append(data["coordsLocked"])
            self.coordinates.append(thisSystem)
        df=pd.DataFrame(self.coordinates,columns=['System',"x","y","z","locked"])
        #print(df)
        fig = px.scatter_3d(data_frame=df, x='x', y='z', z='y',color='System')
        fig.show()
        
        return self.coordinates

In [9]:
from datetime import datetime
date_posted = '2014-01-15T01:35:30.314Z'
datetime.strptime(date_posted, '%Y-%m-%dT%H:%M:%S.%fZ')
class factionHistory:
    def __init__(self,system):
        self.system=system
        self.history=[]
    def getHistory(self):
        endpoint="https://www.edsm.net/api-system-v1/factions"
        PARAMS = {"systemName":self.system,"showHistory":1,"isPlayer":1} 
        r = requests.get(url=endpoint, params = PARAMS) 
        data = r.json()
        #print(data)
        
        for item in data["factions"]:
            #print(item["influenceHistory"])
            
            for tick in item["influenceHistory"].items():
                thisData=[]
                thisData.append(item["name"].strip())
                ts=int(tick[0])
                #thisData.append(ts)
                thisData.append(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d'))
                thisData.append(float(tick[1])*100)
                thisData.append(ts)
                self.history.append(thisData)
            #print(len(thisData))
        #print(len(self.history))
        #for item in self.history:
            #print(item)
        df=pd.DataFrame.from_records(self.history,columns=['faction',"date","influence","ts"])
        #df.drop("Pilots' Federation Local Branch",axis=1)
        #filter=df["influence"] > 0.0
        #df=df[filter]
        df=df.sort_values(by=["ts"])
        df=df.set_index("ts")
        
        #with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        #    print(df)
        
        fig = px.line(df, x="date", y="influence", color="faction")
        fig.show()


In [10]:
def graphFaction(faction):
    
    ptl=factionLocations(referenceFaction=faction)
    systems=ptl.listFactionLocations()
    x=factionCoordinates(faction,systems)
    x.gatherCoordinates()
    
    
    for system in systems:
        plotter=factionInfluence(system=system,graph=1)
        plotter.getInf()
        history=factionHistory(system)
        history.getHistory()

In [ ]:
text=widgets.Text(
    value='Enter a Faction and Press Enter',
    placeholder='Enter Factions Here',
    description='Faction:',
    disabled=False
)
display(text)


def handle_submit(sender):
    faction=text.value
    graphFaction(faction)
text.on_submit(handle_submit)